# Galaxy Neighbor Analysis — Demo

This notebook demonstrates how to use `galaxy_neighbors.py` to find faint galaxy neighbors around bright galaxies across a grid of magnitude limits, for both fiducial and stochastic models.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from galaxy_neighbors import AnalysisConfig, GalaxyModel, run_neighbor_analysis

plt.style.use('seaborn-v0_8-ticks')
plt.rcParams.update({'font.size': 16, 'xtick.top': True, 'ytick.right': True,
                     'xtick.direction': 'in', 'ytick.direction': 'in'})

## 1. Configuration

All magnitude limits, redshift, and survey area live in one place.
Change them here and everything downstream updates automatically.

In [ ]:
cfg = AnalysisConfig(
    bright_limits=[-22.0, -21.5, -21.0],      # MUV < these => 'bright'
    faint_limits=[-17.5, -17.75, -18.0,
                  -18.25, -18.5, -18.75],      # MUV < these => 'faint neighbor'
    preselect_faint_limit=-17.5,               # coarse pre-cut (>= brightest faint limit)
    redshift=10.145,
    survey_area_arcmin2=12.24,                 # NIRSpec MSA
)

print(f"Search box half-side: {cfg.search_box_mpc:.3f} Mpc")
print(f"Bright keys: {cfg.bright_names}")
print(f"Faint keys:  {cfg.faint_names}")

## 2. Load models and run the analysis

Both models go through the same code path — only the input files differ.

In [ ]:
# --- file paths (edit these) ---
HALO_CATALOG = '/lustre/astro/ivannik/21cmFAST_cache/d12b21e80b7885d62d31717c2c2d8421/1952/ffa852ccaa39d8f82951cc98ff798ab4/10.5000/HaloCatalog.h5'
MUV_FIDUCIAL  = '/lustre/astro/ivannik/catalog_fiducial_bigger_new_save.h5'
MUV_STOCH     = '/lustre/astro/ivannik/catalog_stoch_bigger_new3.h5'

# Both models share the same halo catalog; only MUV assignments differ
results_fid, results_stoc = run_neighbor_analysis(
    fiducial_halo_path=HALO_CATALOG,
    fiducial_muv_path=MUV_FIDUCIAL,
    stochastic_halo_path=HALO_CATALOG,
    stochastic_muv_path=MUV_STOCH,
    config=cfg,
    muv_index=0,
)

## 3. Inspect results

Results are nested dicts: `results[bright_key][faint_key]` gives a **list of `NeighborResult` objects**, one per bright galaxy.

In [ ]:
# Quick summary: number of bright galaxies per threshold
for bkey in cfg.bright_names:
    fkey = cfg.faint_names[0]  # use the first faint limit
    n_bright = len(results_fid[bkey][fkey])
    avg_neighbors = np.mean([r.n_neighbors for r in results_fid[bkey][fkey]])
    print(f"[fiducial] bright_limit={bkey}: {n_bright} bright galaxies, "
          f"avg neighbors (faint_limit={fkey}) = {avg_neighbors:.1f}")

In [ ]:
# Access a single result
example = results_fid['M22']['M17.5'][0]
print(example)
print(f"  Bright galaxy mag:    {example.bright_mag:.2f}")
print(f"  Bright galaxy coords: {example.bright_coord}")
print(f"  Faint neighbor mags:  {example.faint_mags[:5]} ...")
print(f"  Distances [Mpc]:      {example.distances[:5]} ...")

## 4. Example plot: mean neighbor count vs faint magnitude limit

For a given bright threshold, compare fiducial vs stochastic.

In [ ]:
def mean_neighbor_counts(results, bright_key, faint_names):
    """Compute mean and std of neighbor count across bright galaxies."""
    means, stds = [], []
    for fkey in faint_names:
        counts = [r.n_neighbors for r in results[bright_key][fkey]]
        means.append(np.mean(counts))
        stds.append(np.std(counts))
    return np.array(means), np.array(stds)


fig, axes = plt.subplots(1, len(cfg.bright_names), figsize=(5 * len(cfg.bright_names), 5),
                         sharey=True)

faint_vals = np.array(cfg.faint_limits)

for ax, bkey, blim in zip(axes, cfg.bright_names, cfg.bright_limits):
    for results, label, color in [
        (results_fid,  'fiducial',    'steelblue'),
        (results_stoc, 'stochastic',  'tomato'),
    ]:
        means, stds = mean_neighbor_counts(results, bkey, cfg.faint_names)
        ax.errorbar(faint_vals, means, yerr=stds, label=label,
                    color=color, marker='o', capsize=4)

    ax.set_title(f'Bright limit: MUV < {blim}')
    ax.set_xlabel('Faint limit (MUV)')
    ax.invert_xaxis()
    ax.legend()

axes[0].set_ylabel('Mean neighbor count')
fig.tight_layout()
plt.show()

## 5. Example plot: distance distribution for one (bright, faint) pair

In [ ]:
bright_key = 'M21'
faint_key  = 'M18'

def pool_distances(results, bright_key, faint_key):
    """Concatenate distances from all bright galaxies."""
    return np.concatenate(
        [r.distances for r in results[bright_key][faint_key] if r.n_neighbors > 0]
    )

dist_fid  = pool_distances(results_fid,  bright_key, faint_key)
dist_stoc = pool_distances(results_stoc, bright_key, faint_key)

fig, ax = plt.subplots(figsize=(7, 5))
bins = np.linspace(0, max(dist_fid.max(), dist_stoc.max()), 40)
ax.hist(dist_fid,  bins=bins, alpha=0.6, label='fiducial',   color='steelblue')
ax.hist(dist_stoc, bins=bins, alpha=0.6, label='stochastic', color='tomato')
ax.set_xlabel('Distance to bright galaxy [Mpc]')
ax.set_ylabel('Count')
ax.set_title(f'bright < {bright_key} | faint < {faint_key}')
ax.legend()
fig.tight_layout()
plt.show()

## 6. Changing the magnitude grid

To run with different limits, just create a new `AnalysisConfig` and re-run. Nothing else changes.

In [ ]:
cfg_custom = AnalysisConfig(
    bright_limits=[-21.0, -20.0],
    faint_limits=[-17.0, -17.5, -18.0, -19.0],
    preselect_faint_limit=-17.0,
    redshift=10.145,
)

# results_fid2, results_stoc2 = run_neighbor_analysis(..., config=cfg_custom)